In [1]:
#get_ipython().system('pip install pycryptodome==3.14.0')

In [2]:
#!python.exe -m pip install --upgrade pip

In [3]:
import random
import string
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

In [4]:
# 1) AES 클래스 구현 
class AESCipher:
    # mode는 정수형태의 상수로 구성
    MODE_ECB = 1  # ECB 모드
    MODE_CBC = 2  # CBC 모드
    MODE_OFB = 3  # OFB 모드
    
    def __init__(self):
        self.blocksize = 16  # 블록 사이즈
        self.mode = AES.MODE_ECB  # 기본 모드로 ECB 설정

    def Set_Mode(self, mode):
        # 주어진 모드에 따라 암호화 모드 설정
        if mode == self.MODE_ECB:
            self.mode = AES.MODE_ECB
        elif mode == self.MODE_CBC:
            self.mode = AES.MODE_CBC
        elif mode == self.MODE_OFB:
            self.mode = AES.MODE_OFB
        else:
            # 유효하지 않은 모드일 경우 오류 발생
            raise ValueError("잘못된 AES 모드입니다.")

    def Create_IV(self):
        # 초기화 벡터 생성 : IV는 숫자, 영문자 대소문자로 구성된 16바이트 String
        return ''.join(random.choices(string.ascii_letters + string.digits, k=self.blocksize))

    def Create_Key(self, key_size=16):
        # key는 16/24/32 바이트 값만 지원
        if key_size not in [16, 24, 32]:
            raise ValueError("잘못된 Key size입니다.")
        # 키 생성
        return ''.join(random.choices(string.ascii_letters + string.digits, k=key_size))

    def pad(self, s):
        # 블록 사이즈에 맞게 패딩
        return s + (self.blocksize - len(s) % self.blocksize) * chr(self.blocksize - len(s) % self.blocksize).encode()

    def unpad(self, s):
        # 패딩 제거
        return s[:-ord(s[len(s) - 1:])]

    def Encrypt(self, plain_text, key, iv=None):
        # CBC나 OFB 모드에서 IV가 필요한 경우 검증
        if self.mode != AES.MODE_ECB and iv is None:
            raise ValueError("CBC 및 OFB 모드에는 IV가 필요합니다.")

        data = self.pad(plain_text.encode())
        
        # ECB 모드일 경우
        if self.mode == AES.MODE_ECB:
            cipher = AES.new(key.encode(), self.mode)
            encrypted = cipher.encrypt(data)
        else:  # CBC나 OFB 모드일 경우
            cipher = AES.new(key.encode(), self.mode, iv.encode())
            encrypted = cipher.encrypt(data)

        # 결과 반환 : cipher_text를 return
        return encrypted.hex()

    def Decrypt(self, cipher_text, key, iv=None):
        # CBC나 OFB 모드에서 IV가 필요한 경우 검증
        if self.mode != AES.MODE_ECB and iv is None:
            raise ValueError("CBC 및 OFB 모드에는 IV가 필요합니다.")

        enc_data = bytes.fromhex(cipher_text)
        
        # ECB 모드일 경우
        if self.mode == AES.MODE_ECB:
            cipher = AES.new(key.encode(), self.mode)
            decrypted = cipher.decrypt(enc_data)
        else:  # CBC나 OFB 모드일 경우
            cipher = AES.new(key.encode(), self.mode, iv.encode())
            decrypted = cipher.decrypt(enc_data)

        # 결과 반환: 복호화된 plain text를 return
        return self.unpad(decrypted).decode()
        

---
# ✅**AES Test**

In [5]:
# plaintext는 사용자로부터 string 형태로 받아옴
plaintext = input("평문을 입력하세요: ")

aes = AESCipher()

# AES 모드를 정수형태의 상수로 사용자로부터 받아옴, mode 숫자 잘못 선택하면 ValueError 발생
aes_mode_input = input(f"AES 모드를 선택하세요 (1: ECB, 2: CBC, 3: OFB): ")
aes_mode = int(aes_mode_input) if aes_mode_input else 1 # 아무 입력도 없으면 자동으로 1을 선택
aes.Set_Mode(aes_mode)

# key_size는 사용자로부터 숫자로 받아옴, key_size는 16, 24, 32 중에서 선택할 수 있음
key_size_input = input("Key size를 선택하세요 (16/24/32 중 하나): ")
key_size = int(key_size_input) if key_size_input else 16 # 아무 입력도 없으면 자동으로 16을 선택
while key_size not in [16, 24, 32]:  # 유효한 값인지 확인
    print("잘못된 Key size입니다. 다시 입력하세요.")
    key_size_input = input("Key size를 선택하세요 (16/24/32 중 하나): ")
    key_size = int(key_size_input) if key_size_input else 16

# 키를 생성하고, 초기화 벡터를 생성
key = aes.Create_Key(key_size)
iv = aes.Create_IV()

# 평문을 암호화하고 그 결과를 출력
encrypted_data = aes.Encrypt(plaintext, key, iv)
#print("암호화 결과 :", encrypted_data)

# 암호문을 복호화
decrypted_data = aes.Decrypt(encrypted_data, key, iv)

# 복호화된 텍스트가 비어있는지 확인하고, 그에 따라 적절한 메시지를 출력
if not decrypted_data.strip():  # strip()을 사용하여 공백만 있는 문자열도 체크
    print("복호화 결과 :", "빈 평문입니다.")
else:
    print("복호화 결과 :", decrypted_data)


평문을 입력하세요:  Hello, World!
AES 모드를 선택하세요 (1: ECB, 2: CBC, 3: OFB):  1
Key size를 선택하세요 (16/24/32 중 하나):  16


복호화 결과 : Hello, World!


---
# **AES Auto-test**

In [6]:
# Mode 1, 2, 3 모두에서 key size 16, 24, 32 모두 테스트에 통과함
# AES 알고리즘이 지원하는 Key size들에 대해 다양한 모드에서 정상 동작한다는 것을 확인
def test_aes_modes_with_different_key_sizes():
    print("\n[+] Testing AES encryption/decryption across different modes and key sizes")

    plaintext = "Test data for AES encryption"
    aes = AESCipher()

    for mode in [AESCipher.MODE_ECB, AESCipher.MODE_CBC, AESCipher.MODE_OFB]:
        aes.Set_Mode(mode)
        for key_size in [16, 24, 32]:
            key = aes.Create_Key(key_size)
            iv = None
            if mode != AESCipher.MODE_ECB:
                iv = aes.Create_IV()

            encrypted_data = aes.Encrypt(plaintext, key, iv)
            decrypted_data = aes.Decrypt(encrypted_data, key, iv)

            assert decrypted_data == plaintext
            print(f"Mode {mode} with key size {key_size}: PASSED") # 정상적인 결과

# 예상대로 실패함. Key size가 요구사항에 맞지 않는 숫자 일 경우, 암호화/복호화를 시도하지 않음.
def test_aes_invalid_key_size():
    print("\n[+] Testing AES with invalid key size")

    aes = AESCipher()
    try:
        key = aes.Create_Key(15)
        print("Invalid key size test: UNEXPETED PASSED") # 비정상적인 결과
    except ValueError:
        print("Invalid key size test: EXPECTED FAILED") # 정상적인 결과

# 예상대로 실패함. CBC와 OFB 모드에서는 초기화벡터(IV)가 필수임.
def test_aes_without_iv_in_cbc_mode():
    print("\n[+] Testing AES CBC mode without IV")

    plaintext = "Test data for AES CBC mode"
    aes = AESCipher()
    aes.Set_Mode(AESCipher.MODE_CBC)
    key = aes.Create_Key(16)

    try:
        encrypted_data = aes.Encrypt(plaintext, key)
        print("AES CBC without IV test: UNEXPETED PASSED") # 비정상적인 결과
    except ValueError:
        print("AES CBC without IV test: EXPECTED FAILED") # 정상적인 결과

def test_aes_without_iv_in_ofb_mode():
    print("\n[+] Testing AES OFB mode without IV")

    plaintext = "Test data for AES OFB mode"
    aes = AESCipher()
    aes.Set_Mode(AESCipher.MODE_OFB)
    key = aes.Create_Key(16)

    try:
        encrypted_data = aes.Encrypt(plaintext, key)
        print("AES OFB without IV test: UNEXPETED PASSED") # 비정상적인 결과
    except ValueError:
        print("AES OFB without IV test: EXPECTED FAILED") # 정상적인 결과

# ECB 모드로 선택됐을 때는 암호화, 복호화 과정에서 IV를 사용하지 않음. 따라서 IV를 입력해도 아무 의미 없음.
def test_aes_with_iv_in_ecb_mode():
    print("\n[+] Testing AES ECB mode with IV")

    plaintext = "Test data for AES ECB mode"
    aes = AESCipher()
    aes.Set_Mode(AESCipher.MODE_ECB)
    key = aes.Create_Key(16)

    try:
        encrypted_data = aes.Encrypt(plaintext, key)
        print("AES ECB with IV test: PASSED") # 비정상적인 결과
    except ValueError:
        print("AES ECB with IV test: FAILED") # 정상적인 결과

# 긴 문자열, 특수 문자 및 유니코드 문자를 포함한 AES 테스트
# 모든 케이스에서 통과했음. 이는 AES가 다양한 문자 및 데이터 유형에 대해 올바르게 작동한다는 것을 뜻함.
def test_aes_with_long_string():
    print("\n[+] Testing AES with long string")

    plaintext = "A" * 1000
    aes = AESCipher()
    aes.Set_Mode(AESCipher.MODE_CBC)
    key = aes.Create_Key(16)
    iv = aes.Create_IV()

    encrypted_data = aes.Encrypt(plaintext, key, iv)
    decrypted_data = aes.Decrypt(encrypted_data, key, iv)

    assert decrypted_data == plaintext
    print("AES with long string: PASSED") # 정상적인 결과

def test_aes_with_special_characters():
    print("\n[+] Testing AES with special characters")

    plaintext = "!@#$%^&*()_+-=[]{}|;:',.<>?/"
    aes = AESCipher()
    aes.Set_Mode(AESCipher.MODE_CBC)
    key = aes.Create_Key(16)
    iv = aes.Create_IV()

    encrypted_data = aes.Encrypt(plaintext, key, iv)
    decrypted_data = aes.Decrypt(encrypted_data, key, iv)

    assert decrypted_data == plaintext
    print("AES with special characters: PASSED") # 정상적인 결과

def test_aes_with_unicode_characters():
    print("\n[+] Testing AES with unicode characters")

    plaintext = "안녕하세요, 테스트 데이터입니다."
    aes = AESCipher()
    aes.Set_Mode(AESCipher.MODE_CBC)
    key = aes.Create_Key(16)
    iv = aes.Create_IV()

    encrypted_data = aes.Encrypt(plaintext, key, iv)
    decrypted_data = aes.Decrypt(encrypted_data, key, iv)

    assert decrypted_data == plaintext
    print("AES with unicode characters: PASSED") # 정상적인 결과

# 잘못된 키 또는 IV를 사용하여 AES 복호화 테스트
# 두 케이스 모두 예상대로 실패함. 잘못된 키 또는 IV를 사용할 때 복호화를 시도하면 잘못된 결과를 초래함.
def test_aes_incorrect_key():
    print("\n[+] Testing AES decryption with incorrect key")

    plaintext = "Test data for AES encryption"
    aes = AESCipher()
    aes.Set_Mode(AESCipher.MODE_CBC)
    key = aes.Create_Key(16)
    incorrect_key = aes.Create_Key(16)
    iv = aes.Create_IV()

    encrypted_data = aes.Encrypt(plaintext, key, iv)
    
    try:
        decrypted_data = aes.Decrypt(encrypted_data, incorrect_key, iv)
        if decrypted_data != plaintext:
            print("AES decryption with incorrect key: EXPECTED FAILED") # 정상적인 결과
        else:
            print("AES decryption with incorrect key: UNEXPETED PASSED") # 비정상적인 결과
    except UnicodeDecodeError:
        print("UnicodeDecodeError")

def test_aes_incorrect_iv():
    print("\n[+] Testing AES decryption with incorrect IV")

    plaintext = "Test data for AES encryption"
    aes = AESCipher()
    aes.Set_Mode(AESCipher.MODE_CBC)
    key = aes.Create_Key(16)
    iv = aes.Create_IV()
    incorrect_iv = aes.Create_IV()

    encrypted_data = aes.Encrypt(plaintext, key, iv)
    
    try:
        decrypted_data = aes.Decrypt(encrypted_data, key, incorrect_iv)
        if decrypted_data != plaintext:
            print("AES decryption with incorrect IV: EXPECTED FAILED") # 정상적인 결과
        else:
            print("AES decryption with incorrect IV: UNEXPETED PASSED") # 정상적인 결과
    except UnicodeDecodeError:
        print("UnicodeDecodeError")

# 빈 평문으로 AES 테스트: 평문이 없을 때도 AES가 올바르게 작동함
def test_aes_empty_plaintext():
    print("\n[+] Testing AES with empty plaintext")

    plaintext = ""
    aes = AESCipher()
    aes.Set_Mode(AESCipher.MODE_CBC)
    key = aes.Create_Key(16)
    iv = aes.Create_IV()

    encrypted_data = aes.Encrypt(plaintext, key, iv)
    decrypted_data = aes.Decrypt(encrypted_data, key, iv)

    assert decrypted_data == plaintext
    print("AES with empty plaintext: PASSED") # 정상적인 결과


# 동일한 키 및 IV를 여러 번 사용하여 AES 테스트
# 여러 번의 암호화 및 복호화를 거쳐도 동일한 결과가 나와야함.
def test_aes_same_key_iv_multiple_times():
    print("\n[+] Testing AES with same key and IV multiple times")

    plaintext1 = "First Test data for AES encryption"
    plaintext2 = "Second Test data for AES encryption"
    aes = AESCipher()
    aes.Set_Mode(AESCipher.MODE_CBC)
    key = aes.Create_Key(16)
    iv = aes.Create_IV()

    encrypted_data1 = aes.Encrypt(plaintext1, key, iv)
    encrypted_data2 = aes.Encrypt(plaintext2, key, iv)

    decrypted_data1 = aes.Decrypt(encrypted_data1, key, iv)
    decrypted_data2 = aes.Decrypt(encrypted_data2, key, iv)

    assert decrypted_data1 == plaintext1
    assert decrypted_data2 == plaintext2
    print("AES with same key and IV multiple times: PASSED") # 정상적인 결과


In [7]:
test_aes_modes_with_different_key_sizes()
test_aes_invalid_key_size()
test_aes_without_iv_in_cbc_mode()
test_aes_without_iv_in_ofb_mode()
test_aes_with_iv_in_ecb_mode()
test_aes_with_long_string()
test_aes_with_special_characters()
test_aes_with_unicode_characters()
test_aes_incorrect_key()
test_aes_incorrect_iv()
test_aes_empty_plaintext()
test_aes_same_key_iv_multiple_times()


[+] Testing AES encryption/decryption across different modes and key sizes
Mode 1 with key size 16: PASSED
Mode 1 with key size 24: PASSED
Mode 1 with key size 32: PASSED
Mode 2 with key size 16: PASSED
Mode 2 with key size 24: PASSED
Mode 2 with key size 32: PASSED
Mode 3 with key size 16: PASSED
Mode 3 with key size 24: PASSED
Mode 3 with key size 32: PASSED

[+] Testing AES with invalid key size
Invalid key size test: EXPECTED FAILED

[+] Testing AES CBC mode without IV
AES CBC without IV test: EXPECTED FAILED

[+] Testing AES OFB mode without IV
AES OFB without IV test: EXPECTED FAILED

[+] Testing AES ECB mode with IV
AES ECB with IV test: PASSED

[+] Testing AES with long string
AES with long string: PASSED

[+] Testing AES with special characters
AES with special characters: PASSED

[+] Testing AES with unicode characters
AES with unicode characters: PASSED

[+] Testing AES decryption with incorrect key
AES decryption with incorrect key: EXPECTED FAILED

[+] Testing AES decrypt